In [22]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from joblib import dump, load
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [23]:
des_arr = np.array([])
for image in os.listdir('train'):
    path = os.path.join('train',image)
    img = cv.imread(path)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    _, des = sift.detectAndCompute(img,None)
    des_arr = des if des_arr.size == 0 else np.vstack((des_arr, des))

In [24]:
kmeans = KMeans(n_clusters=512, random_state=0).fit(des_arr)

c:\Users\nedor\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [25]:
dump(kmeans, 'kmeans.joblib') 

['kmeans.joblib']

In [44]:
def vectorize_image(image:np.array,model,n_clusters=512):
    _,des = sift.detectAndCompute(image,None)
    classes = model.predict(des)
    hist = np.zeros(n_clusters)
    for clss in classes:
        hist[clss] += 1

    hist /= len(classes)
    return hist

In [15]:
# import shutil
# import random

# source_folder = r'D:\LabsCV\lab13\Search-Similar-Images\VOCdevkit\VOC2012\JPEGImages'
# destination_folder = r'D:\LabsCV\lab13\Search-Similar-Images\db'

# if not os.path.exists(destination_folder):
#     os.makedirs(destination_folder)

# # Получаем список всех файлов в исходной директории
# all_files = os.listdir(source_folder)

# # Выбираем случайные 1000 файлов
# selected_files = random.sample(all_files, min(1000, len(all_files)))

# # Копируем выбранные файлы в новую директорию
# for file_name in selected_files:
#     source_path = os.path.join(source_folder, file_name)
#     destination_path = os.path.join(destination_folder, file_name)
#     shutil.copyfile(source_path, destination_path)

# print("Создана новая директория с 1000 изображениями:", destination_folder)

Создана новая директория с 1000 изображениями: D:\LabsCV\lab13\Search-Similar-Images\db


In [41]:
db_csv_name = 'db.csv'

In [52]:
df_list = [
    {
        'uid': ind + 1,
        'filepath': os.path.join('db', image),
        'encoding_vector': vectorize_image(cv.imread(os.path.join('db', image)), kmeans)
    }
    for ind, image in enumerate(os.listdir('db'))
]

df = pd.DataFrame(df_list)
df['encoding_vector'] = df['encoding_vector'].apply(lambda x: np.array2string(x, separator=','))
df.to_csv(db_csv_name, mode='a', header=not os.path.exists(db_csv_name), sep='\t', index=False)

In [53]:
db = pd.read_csv('db.csv',delimiter='\t')
db['encoding_vector'] = db['encoding_vector'].apply(lambda x: eval(x))

In [54]:
X = np.array(db['encoding_vector'].tolist())
nbrs = NearestNeighbors(n_neighbors=5, metric='cosine').fit(X)